In [4]:
import tensorflow.keras
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Nadam
from tensorflow.keras.utils import to_categorical
from tensorflow import keras

In [5]:
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

train_images = train_images.reshape((50000, 32 * 32 * 3))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 32 * 32 * 3))
test_images = test_images.astype("float32") / 255

In [6]:
#List of 4 optimizers
optimizers = { 
    "Adam": Adam,
    "SGD": SGD,
    "RMSProp": RMSprop,
    "Nadam": Nadam
}

#List of 2 loss functions
loss_functions = ["sparse_categorical_crossentropy", "categorical_crossentropy"]

#List of 3 learning rates
learning_rates = [0.01, 0.001, 0.0001] 

#List of 2 epochs
epochs = [5, 15]


#Loops through all the optimizers, loss functions, learning rates, and epochs creating a model for each combination and recording its accuracy.
results = []
for opt_name, opt_class in optimizers.items(): 
    for loss in loss_functions:
        for rate in learning_rates:
            for epoch in epochs:
                print(f"\nLoading ---- Optimizer: {opt_name}, Loss: {loss}, Learning Rate: {rate}, Epochs: {epoch}")

                model = keras.Sequential([
                    layers.Dense(512, activation="relu", input_shape=(32 * 32 * 3,)),
                    layers.Dense(10, activation="softmax")
                ])

                #If the loss function is 'categorical', convert class vector into one-hot encoded matrix
                if loss == "categorical_crossentropy":
                    train_labels_encoded = to_categorical(train_labels, num_classes=10)
                    test_labels_encoded = to_categorical(test_labels, num_classes=10)
                else:
                    train_labels_encoded = train_labels
                    test_labels_encoded = test_labels

                optimizer = opt_class(learning_rate=rate)

                model.compile(optimizer=optimizer,#Creates Model with given optimizer, loss function, and learning rate
                            loss=loss,
                            metrics=["accuracy"]) #Metric is accuracy

                model.fit(train_images, train_labels_encoded, epochs=epoch, batch_size=128) #Fit using set epochs

                test_loss, test_acc = model.evaluate(test_images, test_labels_encoded) #Evaluate model on test data
                train_loss, train_acc = model.evaluate(train_images, train_labels_encoded) #Evaluate model on train data

                #Creates a dictionary with the results and appends it to the results list
                results.append({
                    "Optimizer": opt_name,
                    "Loss Function": loss,
                    "Learning Rate": rate,
                    "Epochs": epoch,
                    "Test Accuracy": test_acc,
                    "Train Accuracy": train_acc

                })

#Converts the results list to a DataFrame and saves it to a CSV file so I don't have to run this again.
import pandas as pd
results_df = pd.DataFrame(results)
results_df.to_csv('resultsDL3.csv', index=False)

#Total result is 48 different combinations (4 optimizers * 2 loss functions * 3 learning rates * 2 epochs)


Loading ---- Optimizer: Adam, Loss: sparse_categorical_crossentropy, Learning Rate: 0.01, Epochs: 5


C:\Users\Sean\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 7s 17ms/step - accuracy: 0.1507 - loss: 9.9071
Epoch 2/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 7s 18ms/step - accuracy: 0.1827 - loss: 2.0749
Epoch 3/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 7s 17ms/step - accuracy: 0.1861 - loss: 2.0580
Epoch 4/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 6s 17ms/step - accuracy: 0.1851 - loss: 2.0532
Epoch 5/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - accuracy: 0.1871 - loss: 2.0465
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1974 - loss: 2.0466
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.1925 - loss: 2.0441

Loading ---- Optimizer: Adam, Loss: sparse_categorical_crossentropy, Learning Rate: 0.01, Epochs: 15
Epoch 1/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 8s 18ms/step - accuracy: 0.1493 - loss: 10.8323
Epoch 2/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 7s 17ms/step - accuracy: 0.2346 - loss: 1.9986
Epoch 3/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 7s 17ms/step - accuracy: 0.2447 - loss: 1.9812
Epoch 4/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step -

In [8]:
import pandas as pd
df = pd.DataFrame(results)
df.to_csv('resultsDL3Finale.csv', index=False)

In [9]:
#The best combination of optimizer, loss function, and learning rate is: (for test accuracy)
print(df.iloc[df['Test Accuracy'].idxmax()])

Optimizer                                   Nadam
Loss Function     sparse_categorical_crossentropy
Learning Rate                              0.0001
Epochs                                         15
Test Accuracy                              0.5047
Train Accuracy                            0.55696
Name: 41, dtype: object
